In [ ]:
!pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 823.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 25.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
!pip install networkx matplotlib

In [ ]:
pip install torch torch-geometric pandas numpy scikit-learn plotly faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 49.3 MB/s eta 0:00:00


# TEMPLATE --- KEEP

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, GATConv
from torch_geometric.data import Data
import plotly.graph_objects as go
import os
from typing import Dict, List, Optional
from sklearn.cluster import KMeans
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import re

# SQLTransformer Class: Handles schema extraction
class SQLTransformer:
    def __init__(self, db_path: str):
        print("Connecting to database...")
        self.db_path = os.path.abspath(db_path)
        self.conn = None
        try:
            self.conn = sqlite3.connect(self.db_path)
            self.cursor = self.conn.cursor()
            print(f"Successfully connected to database at: {self.db_path}")
        except sqlite3.Error as e:
            print(f"Failed to connect to database at {self.db_path}: {e}")
            raise
        self.schema = self._extract_schema()

    def _extract_schema(self) -> Dict:
        print("Extracting schema...")
        schema = {}
        try:
            self.cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tables = self.cursor.fetchall()
            if not tables:
                print("No tables found in the database!")
                return schema
            print("Found tables:", [table[0] for table in tables])
            for table in tables:
                table_name = table[0]
                self.cursor.execute(f"PRAGMA table_info({table_name});")
                columns = self.cursor.fetchall()
                schema[table_name] = [{"name": col[1], "type": col[2], "primary_key": bool(col[5])} for col in columns]
                self.cursor.execute(f"PRAGMA foreign_key_list({table_name});")
                fk_info = self.cursor.fetchall()
                schema[table_name].append({"foreign_keys": [{"from": fk[3], "to_table": fk[2], "to_column": fk[4]} for fk in fk_info]})
        except sqlite3.Error as e:
            print(f"Error extracting schema: {e}")
            raise
        return schema

    def close(self):
        if self.conn:
            self.conn.close()

# GraphSAGE Model with GAT and Edge Features
class GraphSAGE(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, embedding_dim, output_dim, edge_dim=1, heads=2):
        super(GraphSAGE, self).__init__()
        # First layer: GATConv with attention and edge features
        self.conv1 = GATConv(input_dim, hidden_dim, heads=heads, edge_dim=edge_dim, dropout=0.2)
        # Second layer: SAGEConv to aggregate with mean (simpler aggregation)
        self.conv2 = SAGEConv(hidden_dim * heads, embedding_dim, aggr="mean")
        self.predictor = torch.nn.Linear(embedding_dim, output_dim)
        self.dropout = torch.nn.Dropout(p=0.2)  # Dropout for regularization

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        # First layer with attention
        x = self.conv1(x, edge_index, edge_attr)
        x = F.elu(x)  # ELU for non-linearity (alternative to ReLU)
        x = self.dropout(x)  # Apply dropout
        # Second layer with mean aggregation
        x = self.conv2(x, edge_index)
        embeddings = F.relu(x)
        predictions = self.predictor(embeddings)
        return predictions, embeddings

# GNNBuilder Class: Dynamically constructs and trains the GNN
class GNNBuilder:
    def __init__(self, sql_transformer: SQLTransformer, analysis_choice: str):
        print("Building GNN based on selected analysis choice...")
        self.sql_transformer = sql_transformer
        self.analysis_choice = analysis_choice.lower()
        self.schema = sql_transformer.schema
        self.graph = self._build_graph()
        self.node_to_idx = {node["id"]: idx for idx, node in enumerate(self.graph["nodes"])}
        self.gnn_data = self._prepare_gnn_data()

    def _infer_relevant_entities(self) -> tuple[Optional[str], Optional[str], Optional[str]]:
        print("Inferring relevant entities for analysis choice:", self.analysis_choice)
        primary, related, transaction = None, None, None

        if "customer" in self.analysis_choice or "segment" in self.analysis_choice:
            primary = next((table for table in self.schema.keys() if "customer" in table.lower()), None)
        elif "product" in self.analysis_choice or "sales" in self.analysis_choice or "item" in self.analysis_choice or "inventory" in self.analysis_choice or "track" in self.analysis_choice:
            primary = next((table for table in self.schema.keys() if "product" in table.lower() or "track" in table.lower() or "item" in table.lower()), None)
        elif "revenue" in self.analysis_choice or "forecast" in self.analysis_choice:
            primary = next((table for table in self.schema.keys() if "invoice" in table.lower() or "order" in table.lower()), None)
        if not primary:
            primary = next((table for table in self.schema.keys() if any(col["primary_key"] for col in self.schema[table][:-1])), None)

        transaction = next((table for table in self.schema.keys() if "invoice" in table.lower() or "order" in table.lower() or "transaction" in table.lower()), None)

        if primary and transaction:
            for table, details in self.schema.items():
                for fk in details[-1]["foreign_keys"]:
                    if fk["to_table"] == primary and table != transaction:
                        related = table
                        break
                    if fk["to_table"] == transaction and table != primary:
                        related = table
                        break

        print(f"Inferred entities - Primary: {primary}, Related: {related}, Transaction: {transaction}")
        return primary, related, transaction

    def _build_graph(self) -> Dict:
        print("Building graph structure...")
        graph = {"nodes": [], "edges": []}
        for table_name in self.schema.keys():
            graph["nodes"].append({"id": table_name, "type": "table"})
            print(f"Added table node: {table_name}")

        primary_entity, related_entity, transaction_entity = self._infer_relevant_entities()
        if not primary_entity:
            print("Could not infer primary entity for the analysis choice. Using default tables.")
            return graph

        primary_key = next((col["name"] for col in self.schema[primary_entity][:-1] if col["primary_key"]), "id")
        try:
            self.sql_transformer.cursor.execute(f"SELECT {primary_key} FROM {primary_entity}")
            primary_rows = self.sql_transformer.cursor.fetchall()
            if not primary_rows:
                print(f"No rows found in {primary_entity}. Skipping node creation.")
            else:
                for row in primary_rows:
                    entity_id = row[0]
                    node_id = f"{primary_entity}_{entity_id}"
                    graph["nodes"].append({"id": node_id, "type": primary_entity.lower(), "entity_id": entity_id})
                    print(f"Added node: {node_id}")
        except sqlite3.Error as e:
            print(f"Error querying {primary_entity}: {e}")
            return graph

        if transaction_entity:
            transaction_key = next((col["name"] for col in self.schema[transaction_entity][:-1] if col["primary_key"]), "id")
            try:
                self.sql_transformer.cursor.execute(f"SELECT {transaction_key} FROM {transaction_entity}")
                transaction_rows = self.sql_transformer.cursor.fetchall()
                if not transaction_rows:
                    print(f"No rows found in {transaction_entity}. Skipping node creation.")
                else:
                    for row in transaction_rows:
                        trans_id = row[0]
                        node_id = f"{transaction_entity}_{trans_id}"
                        graph["nodes"].append({"id": node_id, "type": transaction_entity.lower(), "entity_id": trans_id})
                        print(f"Added node: {node_id}")
            except sqlite3.Error as e:
                print(f"Error querying {transaction_entity}: {e}")

        if related_entity:
            related_key = next((col["name"] for col in self.schema[related_entity][:-1] if col["primary_key"]), "id")
            try:
                self.sql_transformer.cursor.execute(f"SELECT {related_key} FROM {related_entity}")
                related_rows = self.sql_transformer.cursor.fetchall()
                if not related_rows:
                    print(f"No rows found in {related_entity}. Skipping node creation.")
                else:
                    for row in related_rows:
                        rel_id = row[0]
                        node_id = f"{related_entity}_{rel_id}"
                        graph["nodes"].append({"id": node_id, "type": related_entity.lower(), "entity_id": rel_id})
                        print(f"Added node: {node_id}")
            except sqlite3.Error as e:
                print(f"Error querying {related_entity}: {e}")

        edges_added = False
        if transaction_entity:
            for table, details in self.schema.items():
                foreign_keys = details[-1]["foreign_keys"]
                links_to_primary = any(fk["to_table"] == primary_entity for fk in foreign_keys)
                links_to_transaction = any(fk["to_table"] == transaction_entity for fk in foreign_keys)
                if links_to_primary and links_to_transaction and table != primary_entity and table != transaction_entity:
                    print(f"Found linking table: {table}")
                    fk_to_primary = next(fk for fk in foreign_keys if fk["to_table"] == primary_entity)
                    fk_to_transaction = next(fk for fk in foreign_keys if fk["to_table"] == transaction_entity)
                    try:
                        query = f"""
                        SELECT {fk_to_transaction['from']}, {fk_to_primary['from']}
                        FROM {table}
                        WHERE {fk_to_transaction['from']} IS NOT NULL AND {fk_to_primary['from']} IS NOT NULL
                        """
                        self.sql_transformer.cursor.execute(query)
                        edges = self.sql_transformer.cursor.fetchall()
                        if not edges:
                            print(f"No edges found in {table} linking {transaction_entity} to {primary_entity}.")
                        else:
                            for trans_id, entity_id in edges:
                                source_id = f"{transaction_entity}_{trans_id}"
                                target_id = f"{primary_entity}_{entity_id}"
                                if source_id in [node["id"] for node in graph["nodes"]] and target_id in [node["id"] for node in graph["nodes"]]:
                                    graph["edges"].append({"source": source_id, "target": target_id, "type": "link"})
                                    print(f"Added edge: {source_id} -> {target_id}")
                                    edges_added = True
                    except sqlite3.Error as e:
                        print(f"Error creating edges via {table}: {e}")

                if table == transaction_entity:
                    for fk in foreign_keys:
                        if fk["to_table"] == primary_entity:
                            try:
                                # Include edge attribute (e.g., Total from Invoice)
                                self.sql_transformer.cursor.execute(
                                    f"SELECT {fk['from']}, {transaction_key}, Total FROM {transaction_entity} WHERE {fk['from']} IS NOT NULL"
                                )
                                edges = self.sql_transformer.cursor.fetchall()
                                if not edges:
                                    print(f"No direct edges found for {fk['from']} -> {primary_entity} in {transaction_entity}.")
                                else:
                                    for entity_id, trans_id, total in edges:
                                        source_id = f"{transaction_entity}_{trans_id}"
                                        target_id = f"{primary_entity}_{entity_id}"
                                        if source_id in [node["id"] for node in graph["nodes"]] and target_id in [node["id"] for node in graph["nodes"]]:
                                            graph["edges"].append({
                                                "source": source_id,
                                                "target": target_id,
                                                "type": "link",
                                                "total": total if total is not None else 0.0
                                            })
                                            print(f"Added direct edge: {source_id} -> {target_id} with total {total}")
                                            edges_added = True
                            except sqlite3.Error as e:
                                print(f"Error creating direct edges for {fk['from']} -> {primary_entity}: {e}")

        if related_entity and transaction_entity:
            for table, details in self.schema.items():
                for fk in details[-1]["foreign_keys"]:
                    if fk["to_table"] == related_entity and table == transaction_entity:
                        try:
                            self.sql_transformer.cursor.execute(
                                f"SELECT {fk['from']}, {transaction_key} FROM {transaction_entity} WHERE {fk['from']} IS NOT NULL"
                            )
                            edges = self.sql_transformer.cursor.fetchall()
                            if not edges:
                                print(f"No edges found for {fk['from']} -> {related_entity} in {transaction_entity}.")
                            else:
                                for rel_id, trans_id in edges:
                                    source_id = f"{related_entity}_{rel_id}"
                                    target_id = f"{transaction_entity}_{trans_id}"
                                    if source_id in [node["id"] for node in graph["nodes"]] and target_id in [node["id"] for node in graph["nodes"]]:
                                        graph["edges"].append({"source": source_id, "target": target_id, "type": "link"})
                                        print(f"Added edge: {source_id} -> {target_id}")
                                        edges_added = True
                        except sqlite3.Error as e:
                            print(f"Error creating edges for {fk['from']} -> {related_entity}: {e}")

        if not edges_added:
            print("No edges inferred via foreign keys. Adding fallback edges to table nodes...")
            for node in graph["nodes"]:
                if node["type"] != "table":
                    entity_type = node["type"]
                    table_id = next((table for table in self.schema.keys() if table.lower() == entity_type), None)
                    if table_id and node["id"] != table_id:
                        graph["edges"].append({"source": node["id"], "target": table_id, "type": "belongs_to"})
                        print(f"Added fallback edge: {node['id']} -> {table_id}")

        print(f"Graph built with {len(graph['nodes'])} nodes and {len(graph['edges'])} edges.")
        return graph

    def _prepare_gnn_data(self) -> Data:
        print("Preparing GNN data...")
        nodes = [node["id"] for node in self.graph["nodes"]]
        n = len(nodes)
        edge_index = []
        edge_attr = []
        for edge in self.graph["edges"]:
            src_idx = nodes.index(edge["source"])
            tgt_idx = nodes.index(edge["target"])
            edge_index.append([src_idx, tgt_idx])
            # Use 'total' as edge attribute if available, otherwise default to 0
            total = edge.get("total", 0.0)
            edge_attr.append([total])
        edge_index = torch.tensor(edge_index, dtype=torch.long).t()
        edge_attr = torch.tensor(edge_attr, dtype=torch.float) if edge_attr else torch.zeros((edge_index.shape[1], 1), dtype=torch.float)
        features, labels = self._load_enriched_features()
        x = torch.tensor(features, dtype=torch.float) if features is not None else torch.ones((n, 3), dtype=torch.float)
        y = torch.tensor(labels, dtype=torch.float) if labels is not None else None
        return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

    def _load_enriched_features(self) -> tuple[Optional[np.ndarray], Optional[np.ndarray]]:
        print("Loading enriched features...")
        try:
            feature_dim = 3
            features = np.zeros((len(self.graph["nodes"]), feature_dim))
            labels = np.zeros(len(self.graph["nodes"]))

            primary_entity, related_entity, transaction_entity = self._infer_relevant_entities()
            if not primary_entity:
                print("No primary entity identified for feature enrichment.")
                return None, None

            primary_key = next((col["name"] for col in self.schema[primary_entity][:-1] if col["primary_key"]), "id")
            numeric_cols = [col["name"] for col in self.schema[primary_entity][:-1] if col["type"] in ("INTEGER", "REAL", "NUMERIC") and not col["primary_key"]]
            feature_col = numeric_cols[0] if numeric_cols else None

            if transaction_entity and feature_col:
                trans_key = next((col["name"] for col in self.schema[transaction_entity][:-1] if col["primary_key"]), "id")
                for table, details in self.schema.items():
                    foreign_keys = details[-1]["foreign_keys"]
                    links_to_primary = any(fk["to_table"] == primary_entity for fk in foreign_keys)
                    links_to_transaction = any(fk["to_table"] == transaction_entity for fk in foreign_keys)
                    if links_to_primary and links_to_transaction and table != primary_entity and table != transaction_entity:
                        fk_to_primary = next(fk for fk in foreign_keys if fk["to_table"] == primary_entity)
                        fk_to_transaction = next(fk for fk in foreign_keys if fk["to_table"] == transaction_entity)
                        numeric_cols = [col["name"] for col in self.schema[table][:-1] if col["type"] in ("INTEGER", "REAL", "NUMERIC") and col["name"] not in (fk_to_primary["from"], fk_to_transaction["from"])]
                        agg_col = numeric_cols[0] if numeric_cols else None
                        if agg_col:
                            try:
                                query = f"""
                                SELECT {fk_to_primary['from']}, SUM({agg_col}) AS TotalValue
                                FROM {table}
                                GROUP BY {fk_to_primary['from']}
                                """
                                df = pd.read_sql_query(query, self.sql_transformer.conn)
                                for node in self.graph["nodes"]:
                                    if node["type"] == primary_entity.lower():
                                        entity_id = node["entity_id"]
                                        idx = self.node_to_idx[node["id"]]
                                        row = df[df[fk_to_primary["from"]] == entity_id]
                                        if not row.empty:
                                            value = row["TotalValue"].iloc[0] if not pd.isna(row["TotalValue"].iloc[0]) else 0
                                            features[idx] = [value, 0, 0]
                                            labels[idx] = max(0, value)
                            except sqlite3.Error as e:
                                print(f"Error loading features for {primary_entity} via {table}: {e}")

            if related_entity:
                related_key = next((col["name"] for col in self.schema[related_entity][:-1] if col["primary_key"]), "id")
                numeric_cols = [col["name"] for col in self.schema[related_entity][:-1] if col["type"] in ("INTEGER", "REAL", "NUMERIC") and not col["primary_key"]]
                feature_col = numeric_cols[0] if numeric_cols else None
                if feature_col:
                    try:
                        query = f"""
                        SELECT {related_key}, {feature_col}
                        FROM {related_entity}
                        """
                        df = pd.read_sql_query(query, self.sql_transformer.conn)
                        for node in self.graph["nodes"]:
                            if node["type"] == related_entity.lower():
                                entity_id = node["entity_id"]
                                idx = self.node_to_idx[node["id"]]
                                row = df[df[related_key] == entity_id]
                                if not row.empty:
                                    value = row[feature_col].iloc[0] if not pd.isna(row[feature_col].iloc[0]) else 0
                                    features[idx] = [value, 0, 0]
                                    labels[idx] = max(0, value)
                    except sqlite3.Error as e:
                        print(f"Error loading features for {related_entity}: {e}")

            if transaction_entity:
                trans_key = next((col["name"] for col in self.schema[transaction_entity][:-1] if col["primary_key"]), "id")
                numeric_cols = [col["name"] for col in self.schema[transaction_entity][:-1] if col["type"] in ("INTEGER", "REAL", "NUMERIC") and not col["primary_key"]]
                date_cols = [col["name"] for col in self.schema[transaction_entity][:-1] if "date" in col["name"].lower()]
                feature_col = numeric_cols[0] if numeric_cols else None
                date_col = date_cols[0] if date_cols else None
                if feature_col and date_col:
                    try:
                        query = f"""
                        SELECT {trans_key}, {feature_col}, strftime('%Y', {date_col}) AS Year
                        FROM {transaction_entity}
                        """
                        df = pd.read_sql_query(query, self.sql_transformer.conn)
                        for node in self.graph["nodes"]:
                            if node["type"] == transaction_entity.lower():
                                trans_id = node["entity_id"]
                                idx = self.node_to_idx[node["id"]]
                                row = df[df[trans_key] == trans_id]
                                if not row.empty:
                                    value = row[feature_col].iloc[0]
                                    year = float(row["Year"].iloc[0]) if row["Year"].iloc[0] else 0
                                    features[idx] = [value, year, 0]
                                    labels[idx] = max(0, value)
                    except sqlite3.Error as e:
                        print(f"Error loading features for {transaction_entity}: {e}")

            max_features = np.max(features, axis=0, keepdims=True)
            max_features[max_features == 0] = 1
            features = features / max_features

            max_labels = np.max(labels) if np.max(labels) > 0 else 1
            labels = labels / max_labels

            print("Feature Stats:", np.min(features, axis=0), np.max(features, axis=0))
            print("Label Stats:", np.min(labels), np.max(labels))
            return features, labels
        except Exception as e:
            print(f"Error loading enriched features: {e}")
            return None, None

    def train_gnn(self):
        print("Training GraphSAGE with GAT...")
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = GraphSAGE(input_dim=3, hidden_dim=64, embedding_dim=16, output_dim=1, edge_dim=1, heads=2).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)  # Learning rate scheduling
        data = self.gnn_data.to(device)

        if data.y is not None:
            labels = data.y.unsqueeze(1)
            for epoch in range(300):
                model.train()
                optimizer.zero_grad()
                predictions, embeddings = model(data)
                loss = F.mse_loss(predictions, labels)
                loss.backward()
                optimizer.step()
                scheduler.step()  # Update learning rate
                if epoch % 50 == 0:
                    print(f"Epoch {epoch}, Loss: {loss.item()}, LR: {scheduler.get_last_lr()[0]}")
            print(f"Training completed. Final loss: {loss.item()}")
        else:
            print("No labels available; using dummy loss.")
            for epoch in range(300):
                model.train()
                optimizer.zero_grad()
                predictions, embeddings = model(data)
                loss = torch.mean((predictions - data.x) ** 2)
                loss.backward()
                optimizer.step()
                scheduler.step()  # Update learning rate
            print(f"Training completed. Final loss: {loss.item()}")

        model.eval()
        with torch.no_grad():
            predictions, embeddings = model(data)
            predictions = predictions.cpu().numpy().flatten()
            embeddings = embeddings.cpu().numpy()

        predictions_normalized = np.clip(predictions, 0, None)
        max_pred = np.max(predictions_normalized) if np.max(predictions_normalized) > 0 else 1
        predictions_scaled = (predictions_normalized / max_pred) * 100
        return predictions_scaled, embeddings

# ResultsPresenter Class: Visualizes and presents results
class ResultsPresenter:
    def __init__(self, graph: Dict, predictions: np.ndarray, embeddings: np.ndarray, node_to_idx: Dict, analysis_choice: str, sql_transformer: SQLTransformer):
        self.graph = graph
        self.predictions = predictions
        self.embeddings = embeddings
        self.node_to_idx = node_to_idx
        self.analysis_choice = analysis_choice.lower()
        self.sql_transformer = sql_transformer
        self.schema = sql_transformer.schema

        # Initialize FAISS index for efficient similarity search
        embedding_dim = embeddings.shape[1]
        self.index = faiss.IndexFlatL2(embedding_dim)  # L2 distance for similarity
        self.index.add(embeddings.astype(np.float32))  # Add embeddings to the index
        self.node_ids = [node["id"] for node in self.graph["nodes"]]

    def _infer_relevant_entities(self):
        builder = GNNBuilder(self.sql_transformer, self.analysis_choice)
        return builder._infer_relevant_entities()

    def _force_directed_layout(self, nodes, edges, iterations=50):
        """Compute a force-directed layout for nodes and edges."""
        node_ids = [node["id"] for node in nodes]
        pos = {node_id: np.array([np.random.uniform(-1, 1), np.random.uniform(-1, 1)]) for node_id in node_ids}
        k = 1.0 / np.sqrt(len(nodes))  # Optimal distance between nodes

        for _ in range(iterations):
            # Repulsion between all pairs of nodes
            for i, node1 in enumerate(node_ids):
                for j, node2 in enumerate(node_ids):
                    if i != j:
                        delta = pos[node1] - pos[node2]
                        dist = np.linalg.norm(delta)
                        if dist > 0:
                            repulsion = k * k / dist
                            pos[node1] += (delta / dist) * repulsion
                            pos[node2] -= (delta / dist) * repulsion

            # Attraction along edges
            for edge in edges:
                src = edge["source"]
                tgt = edge["target"]
                if src in pos and tgt in pos:
                    delta = pos[src] - pos[tgt]
                    dist = np.linalg.norm(delta)
                    if dist > 0:
                        attraction = (dist * dist) / k
                        pos[src] -= (delta / dist) * attraction * 0.01
                        pos[tgt] += (delta / dist) * attraction * 0.01

        # Normalize positions
        all_pos = np.array(list(pos.values()))
        if len(all_pos) > 0:
            min_pos = all_pos.min(axis=0)
            max_pos = all_pos.max(axis=0)
            if (max_pos - min_pos).max() > 0:
                all_pos = (all_pos - min_pos) / (max_pos - min_pos) * 10 - 5
        return {node_id: all_pos[i] for i, node_id in enumerate(node_ids)}

    def _circular_layout(self, nodes, clusters):
        """Compute a circular layout, grouping nodes by cluster."""
        node_ids = [node["id"] for node in nodes]
        pos = {}
        n_clusters = len(set(clusters))
        radius = 5
        for cluster in range(n_clusters):
            cluster_nodes = [node_ids[i] for i in range(len(node_ids)) if clusters[i] == cluster]
            for idx, node_id in enumerate(cluster_nodes):
                angle = 2 * np.pi * idx / len(cluster_nodes) + (cluster * 2 * np.pi / n_clusters)
                pos[node_id] = np.array([radius * np.cos(angle), radius * np.sin(angle)])
        return pos

    def present_results(self):
        print("Presenting results...")
        primary_entity, _, _ = self._infer_relevant_entities()
        entity_indices = [i for i, node in enumerate(self.graph["nodes"]) if node["type"] == primary_entity.lower()]
        entity_ids = [self.graph["nodes"][i]["id"] for i in entity_indices]
        entity_names = []

        # Fetch names for primary entities with error handling
        if primary_entity and ("predict " in self.analysis_choice or "analyze " in self.analysis_choice):
            primary_key = next((col["name"] for col in self.schema[primary_entity][:-1] if col["primary_key"]), "id")
            name_col = next((col["name"] for col in self.schema[primary_entity][:-1] if "name" in col["name"].lower()), primary_key)
            try:
                query = f"SELECT {primary_key}, {name_col} FROM {primary_entity}"
                df = pd.read_sql_query(query, self.sql_transformer.conn)
                for node in self.graph["nodes"]:
                    if node["type"] == primary_entity.lower():
                        entity_id = node["entity_id"]
                        row = df[df[primary_key] == entity_id]
                        if not row.empty:
                            node["name"] = row[name_col].iloc[0]
                entity_names = [self.graph["nodes"][i].get("name", self.graph["nodes"][i]["id"]) for i in entity_indices]
            except sqlite3.Error as e:
                print(f"Error fetching names for {primary_entity}: {e}")
                entity_names = [self.graph["nodes"][i]["id"] for i in entity_indices]

            # Top 5 Predictions
            entity_predictions = [
                (entity_ids[i], entity_names[i], self.predictions[entity_indices[i]])
                for i in range(len(entity_indices))
            ]
            entity_predictions.sort(key=lambda x: x[2], reverse=True)
            print(f"\nTop 5 Predicted {primary_entity}s:")
            for entity_id, entity_name, scaled_pred in entity_predictions[:5]:
                print(f"{primary_entity}: {entity_id} ({entity_name}), Scaled Prediction: {scaled_pred:.2f}")

            # Visualization 1: Top Customers and Their Invoices
            print("\nGenerating Visualization 1: Top Customers and Their Invoices...")
            try:
                top_entities = [ep[0] for ep in entity_predictions[:5]]
                subgraph_nodes = []
                subgraph_edges = []
                node_types = {}
                node_predictions = {}
                node_names = {}

                # Add top entities
                for idx, node in enumerate(self.graph["nodes"]):
                    if node["id"] in top_entities:
                        subgraph_nodes.append(node)
                        node_types[node["id"]] = node["type"]
                        node_predictions[node["id"]] = self.predictions[idx]
                        node_names[node["id"]] = node.get("name", node["id"])

                # Add related invoices and tracks
                for edge in self.graph["edges"]:
                    if edge["source"] in top_entities or edge["target"] in top_entities:
                        src_node = next(n for n in self.graph["nodes"] if n["id"] == edge["source"])
                        tgt_node = next(n for n in self.graph["nodes"] if n["id"] == edge["target"])
                        if src_node not in subgraph_nodes:
                            subgraph_nodes.append(src_node)
                            idx = self.node_ids.index(src_node["id"])
                            node_types[src_node["id"]] = src_node["type"]
                            node_predictions[src_node["id"]] = self.predictions[idx]
                            node_names[src_node["id"]] = src_node.get("name", src_node["id"])
                        if tgt_node not in subgraph_nodes:
                            subgraph_nodes.append(tgt_node)
                            idx = self.node_ids.index(tgt_node["id"])
                            node_types[tgt_node["id"]] = tgt_node["type"]
                            node_predictions[tgt_node["id"]] = self.predictions[idx]
                            node_names[tgt_node["id"]] = tgt_node.get("name", tgt_node["id"])
                        subgraph_edges.append(edge)

                # Add tracks from InvoiceLine
                invoice_ids = [node["id"] for node in subgraph_nodes if node["type"] == "invoice"]
                for invoice_id in invoice_ids:
                    invoice_num = int(invoice_id.split("_")[1])
                    try:
                        query = f"""
                        SELECT TrackId FROM InvoiceLine WHERE InvoiceId = {invoice_num}
                        """
                        df = pd.read_sql_query(query, self.sql_transformer.conn)
                        for track_id in df["TrackId"]:
                            track_node_id = f"Track_{track_id}"
                            if track_node_id not in [node["id"] for node in subgraph_nodes]:
                                track_node = {"id": track_node_id, "type": "track", "entity_id": track_id}
                                subgraph_nodes.append(track_node)
                                idx = self.node_ids.index(track_node_id) if track_node_id in self.node_ids else len(self.node_ids)
                                node_types[track_node_id] = "track"
                                node_predictions[track_node_id] = self.predictions[idx] if idx < len(self.predictions) else 0
                                # Fetch track name
                                track_query = f"SELECT Name FROM Track WHERE TrackId = {track_id}"
                                track_df = pd.read_sql_query(track_query, self.sql_transformer.conn)
                                track_name = track_df["Name"].iloc[0] if not track_df.empty else track_node_id
                                node_names[track_node_id] = track_name
                            subgraph_edges.append({"source": invoice_id, "target": track_node_id, "type": "contains"})
                    except sqlite3.Error as e:
                        print(f"Error fetching tracks for invoice {invoice_id}: {e}")

                # Compute layout
                pos = self._force_directed_layout(subgraph_nodes, subgraph_edges)
                edge_x = []
                edge_y = []
                for edge in subgraph_edges:
                    src_pos = pos.get(edge["source"])
                    tgt_pos = pos.get(edge["target"])
                    if src_pos is not None and tgt_pos is not None:
                        edge_x.extend([src_pos[0], tgt_pos[0], None])
                        edge_y.extend([src_pos[1], tgt_pos[1], None])

                node_x = []
                node_y = []
                node_text = []
                node_colors = []
                max_pred = max(node_predictions.values()) if node_predictions else 1
                for node in subgraph_nodes:
                    node_id = node["id"]
                    node_pos = pos.get(node_id)
                    if node_pos is not None:
                        node_x.append(node_pos[0])
                        node_y.append(node_pos[1])
                        pred = node_predictions.get(node_id, 0)
                        scaled_pred = (pred / max_pred) * 100 if max_pred > 0 else 0
                        node_text.append(f"{node_names.get(node_id, node_id)} ({node_types.get(node_id).capitalize()})<br>Prediction: {scaled_pred:.2f}")
                        green_value = int(255 * (pred / max_pred)) if max_pred > 0 else 0
                        node_colors.append(f'rgb(0, {green_value}, 0)')

                fig1 = go.Figure()
                fig1.add_trace(go.Scatter(
                    x=edge_x, y=edge_y,
                    line=dict(width=1, color='#888'),
                    hoverinfo='none',
                    mode='lines',
                    name='Edges'
                ))
                fig1.add_trace(go.Scatter(
                    x=node_x, y=node_y,
                    mode='markers+text',
                    hoverinfo='text',
                    marker=dict(size=15, color=node_colors, line=dict(width=1, color='black')),
                    text=[n.split("<br>")[0] for n in node_text],
                    textposition="top center",
                    hovertext=node_text,
                    name='Nodes'
                ))
                fig1.update_layout(
                    title='Top Customers, Their Invoices, and Purchased Tracks',
                    showlegend=True,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    xaxis=dict(showgrid=False, zeroline=False, title=''),
                    yaxis=dict(showgrid=False, zeroline=False, title=''),
                    width=800,
                    height=600
                )
                fig1.show()
            except Exception as e:
                print(f"Error generating Visualization 1: {e}")

        # Visualization 2: Customer Clusters
        if len(entity_indices) > 1:
            print("\nGenerating Visualization 2: Customer Clusters...")
            try:
                entity_embeddings = self.embeddings[entity_indices]
                kmeans = KMeans(n_clusters=min(3, len(entity_indices)), random_state=42)
                entity_clusters = kmeans.fit_predict(entity_embeddings)

                # Build subgraph for customers
                subgraph_nodes = [self.graph["nodes"][i] for i in entity_indices]
                subgraph_edges = []
                similarities = []
                k = min(4, len(entity_indices))
                distances, indices = self.index.search(entity_embeddings, k)
                for i in range(len(entity_indices)):
                    for j in range(1, k):
                        neighbor_global_idx = indices[i][j]
                        if neighbor_global_idx >= 0 and neighbor_global_idx < len(self.embeddings):
                            neighbor_entity_idx = entity_indices.index(neighbor_global_idx) if neighbor_global_idx in entity_indices else -1
                            if neighbor_entity_idx >= 0 and neighbor_entity_idx != i:
                                distance = distances[i][j]
                                similarity = 1 / (1 + distance) if distance > 0 else 0
                                if similarity > 0.5:
                                    subgraph_edges.append({
                                        "source": entity_ids[i],
                                        "target": entity_ids[neighbor_entity_idx],
                                        "type": "similarity",
                                        "similarity": similarity
                                    })
                                    similarities.append(similarity)

                # Compute layout
                pos = self._circular_layout(subgraph_nodes, entity_clusters)
                edge_x = []
                edge_y = []
                edge_text = []
                for edge in subgraph_edges:
                    src_pos = pos.get(edge["source"])
                    tgt_pos = pos.get(edge["target"])
                    if src_pos is not None and tgt_pos is not None:
                        edge_x.extend([src_pos[0], tgt_pos[0], None])
                        edge_y.extend([src_pos[1], tgt_pos[1], None])
                        edge_text.append(f"Similarity: {edge['similarity']:.2f}")

                node_x = []
                node_y = []
                node_text = []
                node_colors = []
                for idx, node in enumerate(subgraph_nodes):
                    node_id = node["id"]
                    node_pos = pos.get(node_id)
                    if node_pos is not None:
                        node_x.append(node_pos[0])
                        node_y.append(node_pos[1])
                        cluster = entity_clusters[idx]
                        node_text.append(f"{entity_names[idx]} ({node_id})<br>Cluster: {cluster}")
                        node_colors.append(['#FF9999', '#66B2FF', '#99FF99'][cluster % 3])

                fig2 = go.Figure()
                fig2.add_trace(go.Scatter(
                    x=edge_x, y=edge_y,
                    line=dict(width=1, color='#888'),
                    hoverinfo='text',
                    hovertext=edge_text,
                    mode='lines',
                    name='Similarity Edges'
                ))
                fig2.add_trace(go.Scatter(
                    x=node_x, y=node_y,
                    mode='markers+text',
                    hoverinfo='text',
                    marker=dict(size=15, color=node_colors, line=dict(width=1, color='black')),
                    text=[n.split("<br>")[0] for n in node_text],
                    textposition="top center",
                    hovertext=node_text,
                    name='Customers'
                ))
                fig2.update_layout(
                    title='Customer Clusters Based on Purchase Behavior',
                    showlegend=True,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    xaxis=dict(showgrid=False, zeroline=False, title=''),
                    yaxis=dict(showgrid=False, zeroline=False, title=''),
                    width=800,
                    height=600
                )
                fig2.show()
            except Exception as e:
                print(f"Error generating Visualization 2: {e}")

            # Visualization 3: Cross-Selling Opportunities
            print("\nGenerating Visualization 3: Cross-Selling Opportunities...")
            try:
                # Build subgraph for top similar pairs
                top_pairs = sorted(subgraph_edges, key=lambda x: x["similarity"], reverse=True)[:5]
                customer_ids = set()
                for pair in top_pairs:
                    customer_ids.add(pair["source"])
                    customer_ids.add(pair["target"])

                subgraph_nodes = [node for node in self.graph["nodes"] if node["id"] in customer_ids]
                subgraph_edges = [edge for edge in top_pairs]
                track_nodes = set()
                for customer_id in customer_ids:
                    customer_num = int(customer_id.split("_")[1])
                    try:
                        query = f"""
                        SELECT il.TrackId, t.Name
                        FROM Invoice i
                        JOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId
                        JOIN Track t ON il.TrackId = t.TrackId
                        WHERE i.CustomerId = {customer_num}
                        """
                        df = pd.read_sql_query(query, self.sql_transformer.conn)
                        for _, row in df.iterrows():
                            track_id = f"Track_{row['TrackId']}"
                            track_nodes.add((track_id, row["Name"]))
                            subgraph_edges.append({"source": customer_id, "target": track_id, "type": "purchased"})
                    except sqlite3.Error as e:
                        print(f"Error fetching tracks for {customer_id}: {e}")

                for track_id, track_name in track_nodes:
                    subgraph_nodes.append({"id": track_id, "type": "track", "name": track_name})

                # Bipartite layout: customers on left, tracks on right
                pos = {}
                customer_y = np.linspace(-5, 5, len(customer_ids))
                for idx, customer_id in enumerate(customer_ids):
                    pos[customer_id] = np.array([-3, customer_y[idx]])
                track_y = np.linspace(-5, 5, len(track_nodes))
                for idx, (track_id, _) in enumerate(track_nodes):
                    pos[track_id] = np.array([3, track_y[idx]])

                edge_x = []
                edge_y = []
                edge_text = []
                for edge in subgraph_edges:
                    src_pos = pos.get(edge["source"])
                    tgt_pos = pos.get(edge["target"])
                    if src_pos is not None and tgt_pos is not None:
                        edge_x.extend([src_pos[0], tgt_pos[0], None])
                        edge_y.extend([src_pos[1], tgt_pos[1], None])
                        if edge["type"] == "similarity":
                            similarity = edge.get("similarity", 0)
                            edge_text.append(f"Similarity: {similarity:.2f}")
                        else:  # "purchased" type
                            edge_text.append("Purchased")

                node_x = []
                node_y = []
                node_text = []
                node_colors = []
                for node in subgraph_nodes:
                    node_id = node["id"]
                    node_pos = pos.get(node_id)
                    if node_pos is not None:
                        node_x.append(node_pos[0])
                        node_y.append(node_pos[1])
                        node_name = node.get("name", node_id)
                        node_text.append(f"{node_name} ({node['type'].capitalize()})")
                        node_colors.append('#FF9999' if node["type"] == "customer" else '#66B2FF')

                fig3 = go.Figure()
                fig3.add_trace(go.Scatter(
                    x=edge_x, y=edge_y,
                    line=dict(width=1, color='#888'),
                    hoverinfo='text',
                    hovertext=edge_text,
                    mode='lines',
                    name='Edges'
                ))
                fig3.add_trace(go.Scatter(
                    x=node_x, y=node_y,
                    mode='markers+text',
                    hoverinfo='text',
                    marker=dict(size=15, color=node_colors, line=dict(width=1, color='black')),
                    text=[n.split("<br>")[0] for n in node_text],
                    textposition="top center",
                    hovertext=node_text,
                    name='Nodes'
                ))
                fig3.update_layout(
                    title='Cross-Selling Opportunities: Customers and Shared Tracks',
                    showlegend=True,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    xaxis=dict(showgrid=False, zeroline=False, title=''),
                    yaxis=dict(showgrid=False, zeroline=False, title=''),
                    width=800,
                    height=600
                )
                fig3.show()
            except Exception as e:
                print(f"Error generating Visualization 3: {e}")

# Main Workflow
if __name__ == "__main__":
    print("Welcome to the Data Insights Generator!")
    db_path = input("Please specify the path to your SQLite database (e.g., Chinook_Sqlite.sqlite): ").strip()
    if not os.path.exists(db_path):
        print("Database not found! Downloading Chinook database as a default...")
        download_path = os.path.join(os.getcwd(), "Chinook_Sqlite.sqlite")
        os.system(f"wget -O {download_path} https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite")
        if not os.path.exists(download_path):
            raise FileNotFoundError(f"Failed to download Chinook database to {download_path}")
        db_path = download_path
        print(f"Chinook database downloaded to: {db_path}")

    # Step 1: Connect and interrogate data source
    sql_transformer = SQLTransformer(db_path)
    print("\nExtracted Schema:", sql_transformer.schema)

    # Step 2: Use LLM to generate analysis choices
    print("Generating analysis options using LLM...")
    model_name = "gpt2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

    table_names = ", ".join(sql_transformer.schema.keys())
    prompt = (
        f"Based on the following database tables: {table_names}, suggest 4 possible data analysis tasks. "
        "Number them 1 to 4 and describe each task briefly. Focus on sales, customer behavior, or trends. "
        "Example: 1. Predict top-selling products based on sales data. 2. Segment customers by purchase history. "
        "3. Analyze revenue trends over time. 4. Identify cross-selling opportunities between products."
    )
    result = generator(prompt, max_length=200, num_return_sequences=1, temperature=0.7)[0]['generated_text']
    print("\nLLM-generated analysis options:")
    print(result)

    # Parse LLM response to extract options using regex
    analysis_choices = []
    pattern = r"(\d+)\.\s*(.+)"
    matches = re.findall(pattern, result)
    for match in matches[:4]:  # Limit to first 4 matches
        number, description = match
        if int(number) <= 4:
            analysis_choices.append(f"{number}. {description.strip()}")
    if not analysis_choices:
        print("Failed to parse LLM response. Falling back to default choices.")
        analysis_choices = [
            "1. Predict top-selling tracks.",
            "2. Segment customers based on purchase behavior.",
            "3. Identify cross-selling opportunities between tracks.",
            "4. Analyze track sales trends."
        ]
    print("\nAvailable analysis options:")
    for choice in analysis_choices:
        print(choice)

    # Let the user select an analysis choice
    analysis_selection = input("Please select an analysis by number (1-4) or press Enter for default (1): ").strip()
    if not analysis_selection or not analysis_selection.isdigit() or int(analysis_selection) > len(analysis_choices) or int(analysis_selection) < 1:
        analysis_selection = "1"

    selected_analysis = next((choice.split(". ", 1)[1] for choice in analysis_choices if choice.startswith(f"{analysis_selection}.")), analysis_choices[0].split(". ", 1)[1])
    print(f"\nSelected Analysis: {selected_analysis}")

    # Step 3: Build and train the GNN
    gnn_builder = GNNBuilder(sql_transformer, selected_analysis)
    predictions_scaled, embeddings = gnn_builder.train_gnn()

    # Step 4: Present results
    presenter = ResultsPresenter(gnn_builder.graph, predictions_scaled, embeddings, gnn_builder.node_to_idx, selected_analysis, sql_transformer)
    presenter.present_results()

    # Clean up
    sql_transformer.close()

Welcome to the Data Insights Generator!
Please specify the path to your SQLite database (e.g., Chinook_Sqlite.sqlite): Chinook_Sqlite.sqlite
Connecting to database...
Successfully connected to database at: /content/Chinook_Sqlite.sqlite
Extracting schema...
Found tables: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']

Extracted Schema: {'Album': [{'name': 'AlbumId', 'type': 'INTEGER', 'primary_key': True}, {'name': 'Title', 'type': 'NVARCHAR(160)', 'primary_key': False}, {'name': 'ArtistId', 'type': 'INTEGER', 'primary_key': False}, {'foreign_keys': [{'from': 'ArtistId', 'to_table': 'Artist', 'to_column': 'ArtistId'}]}], 'Artist': [{'name': 'ArtistId', 'type': 'INTEGER', 'primary_key': True}, {'name': 'Name', 'type': 'NVARCHAR(120)', 'primary_key': False}, {'foreign_keys': []}], 'Customer': [{'name': 'CustomerId', 'type': 'INTEGER', 'primary_key': True}, {'name': 'FirstName', 'type': 'NVARCHAR(40)', 'pri

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Streaming output truncated to the last 5000 lines.
Added node: InvoiceLine_1930
Added node: InvoiceLine_785
Added node: InvoiceLine_1358
Added node: InvoiceLine_213
Added node: InvoiceLine_1931
Added node: InvoiceLine_786
Added node: InvoiceLine_1359
Added node: InvoiceLine_214
Added node: InvoiceLine_1932
Added node: InvoiceLine_787
Added node: InvoiceLine_1360
Added node: InvoiceLine_215
Added node: InvoiceLine_1933
Added node: InvoiceLine_788
Added node: InvoiceLine_1361
Added node: InvoiceLine_216
Added node: InvoiceLine_1934
Added node: InvoiceLine_789
Added node: InvoiceLine_1362
Added node: InvoiceLine_217
Added node: InvoiceLine_1935
Added node: InvoiceLine_790
Added node: InvoiceLine_1363
Added node: InvoiceLine_218
Added node: InvoiceLine_791
Added node: InvoiceLine_1364
Added node: InvoiceLine_1936
Added node: InvoiceLine_219
Added node: InvoiceLine_1937
Added node: InvoiceLine_1938
Added node: InvoiceLine_1939
Added node: InvoiceLine_792
Added node: InvoiceLine_1940
Added n


Generating Visualization 2: Customer Clusters...



Generating Visualization 3: Cross-Selling Opportunities...


PLAY HERE

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, GATConv
from torch_geometric.data import Data
import plotly.graph_objects as go
import os
from typing import Dict, List, Optional
from sklearn.cluster import KMeans
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import re

# SQLTransformer Class: Handles schema extraction
class SQLTransformer:
    def __init__(self, db_path: str):
        print("Connecting to database...")
        self.db_path = os.path.abspath(db_path)
        self.conn = None
        try:
            self.conn = sqlite3.connect(self.db_path)
            self.cursor = self.conn.cursor()
            print(f"Successfully connected to database at: {self.db_path}")
        except sqlite3.Error as e:
            print(f"Failed to connect to database at {self.db_path}: {e}")
            raise
        self.schema = self._extract_schema()

    def _extract_schema(self) -> Dict:
        print("Extracting schema...")
        schema = {}
        try:
            self.cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tables = self.cursor.fetchall()
            if not tables:
                print("No tables found in the database!")
                return schema
            print("Found tables:", [table[0] for table in tables])
            for table in tables:
                table_name = table[0]
                self.cursor.execute(f"PRAGMA table_info({table_name});")
                columns = self.cursor.fetchall()
                schema[table_name] = [{"name": col[1], "type": col[2], "primary_key": bool(col[5])} for col in columns]
                self.cursor.execute(f"PRAGMA foreign_key_list({table_name});")
                fk_info = self.cursor.fetchall()
                schema[table_name].append({"foreign_keys": [{"from": fk[3], "to_table": fk[2], "to_column": fk[4]} for fk in fk_info]})
        except sqlite3.Error as e:
            print(f"Error extracting schema: {e}")
            raise
        return schema

    def close(self):
        if self.conn:
            self.conn.close()

# GraphSAGE Model with GAT and Edge Features
class GraphSAGE(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, embedding_dim, output_dim, edge_dim=2, heads=4):
        super(GraphSAGE, self).__init__()
        # First layer: GATConv with more heads for diverse attention
        self.conv1 = GATConv(input_dim, hidden_dim, heads=heads, edge_dim=edge_dim, dropout=0.3)
        # Second layer: SAGEConv with batch normalization
        self.conv2 = SAGEConv(hidden_dim * heads, embedding_dim, aggr="mean")
        self.bn = torch.nn.BatchNorm1d(embedding_dim)  # Batch normalization
        self.predictor = torch.nn.Linear(embedding_dim, output_dim)
        self.dropout = torch.nn.Dropout(p=0.3)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = self.conv1(x, edge_index, edge_attr)
        x = F.elu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = self.bn(x)  # Apply batch normalization
        embeddings = F.relu(x)
        predictions = self.predictor(embeddings)
        return predictions, embeddings

# GNNBuilder Class: Dynamically constructs and trains the GNN
class GNNBuilder:
    def __init__(self, sql_transformer: SQLTransformer, analysis_choice: str):
        print("Building GNN based on selected analysis choice...")
        self.sql_transformer = sql_transformer
        # Parse only the selected analysis choice
        self.analysis_choice = next((choice.split(". ", 1)[1] for choice in analysis_choice.split("\n") if choice.strip().startswith(str(analysis_choice.split()[0]))), analysis_choice.split(". ", 1)[1])
        print(f"Parsed analysis choice: {self.analysis_choice}")
        self.schema = sql_transformer.schema
        self.graph = self._build_graph()
        self.node_to_idx = {node["id"]: idx for idx, node in enumerate(self.graph["nodes"])}
        self.gnn_data = self._prepare_gnn_data()

    def _infer_relevant_entities(self) -> tuple[Optional[str], Optional[str], Optional[str]]:
        print("Inferring relevant entities for analysis choice:", self.analysis_choice)
        primary, related, transaction = None, None, None

        if "customer" in self.analysis_choice.lower() or "segment" in self.analysis_choice.lower():
            primary = next((table for table in self.schema.keys() if "customer" in table.lower()), None)
        elif "product" in self.analysis_choice.lower() or "sales" in self.analysis_choice.lower() or "item" in self.analysis_choice.lower() or "inventory" in self.analysis_choice.lower() or "track" in self.analysis_choice.lower():
            primary = next((table for table in self.schema.keys() if "product" in table.lower() or "track" in table.lower() or "item" in table.lower()), None)
        elif "revenue" in self.analysis_choice.lower() or "forecast" in self.analysis_choice.lower():
            primary = next((table for table in self.schema.keys() if "invoice" in table.lower() or "order" in table.lower()), None)
        if not primary:
            primary = next((table for table in self.schema.keys() if any(col["primary_key"] for col in self.schema[table][:-1])), None)

        transaction = next((table for table in self.schema.keys() if "invoice" in table.lower() or "order" in table.lower() or "transaction" in table.lower()), None)

        if primary and transaction:
            for table, details in self.schema.items():
                for fk in details[-1]["foreign_keys"]:
                    if fk["to_table"] == primary and table != transaction:
                        related = table
                        break
                    if fk["to_table"] == transaction and table != primary:
                        related = table
                        break

        print(f"Inferred entities - Primary: {primary}, Related: {related}, Transaction: {transaction}")
        return primary, related, transaction

    def _build_graph(self) -> Dict:
        print("Building graph structure...")
        graph = {"nodes": [], "edges": []}
        for table_name in self.schema.keys():
            graph["nodes"].append({"id": table_name, "type": "table"})
            print(f"Added table node: {table_name}")

        primary_entity, related_entity, transaction_entity = self._infer_relevant_entities()
        if not primary_entity:
            print("Could not infer primary entity for the analysis choice. Using default tables.")
            return graph

        primary_key = next((col["name"] for col in self.schema[primary_entity][:-1] if col["primary_key"]), "id")
        try:
            self.sql_transformer.cursor.execute(f"SELECT {primary_key} FROM {primary_entity}")
            primary_rows = self.sql_transformer.cursor.fetchall()
            if not primary_rows:
                print(f"No rows found in {primary_entity}. Skipping node creation.")
            else:
                for row in primary_rows:
                    entity_id = row[0]
                    node_id = f"{primary_entity}_{entity_id}"
                    graph["nodes"].append({"id": node_id, "type": primary_entity.lower(), "entity_id": entity_id})
                    print(f"Added node: {node_id}")
        except sqlite3.Error as e:
            print(f"Error querying {primary_entity}: {e}")
            return graph

        if transaction_entity:
            transaction_key = next((col["name"] for col in self.schema[transaction_entity][:-1] if col["primary_key"]), "id")
            try:
                self.sql_transformer.cursor.execute(f"SELECT {transaction_key} FROM {transaction_entity}")
                transaction_rows = self.sql_transformer.cursor.fetchall()
                if not transaction_rows:
                    print(f"No rows found in {transaction_entity}. Skipping node creation.")
                else:
                    for row in transaction_rows:
                        trans_id = row[0]
                        node_id = f"{transaction_entity}_{trans_id}"
                        graph["nodes"].append({"id": node_id, "type": transaction_entity.lower(), "entity_id": trans_id})
                        print(f"Added node: {node_id}")
            except sqlite3.Error as e:
                print(f"Error querying {transaction_entity}: {e}")

        if related_entity:
            related_key = next((col["name"] for col in self.schema[related_entity][:-1] if col["primary_key"]), "id")
            try:
                self.sql_transformer.cursor.execute(f"SELECT {related_key} FROM {related_entity}")
                related_rows = self.sql_transformer.cursor.fetchall()
                if not related_rows:
                    print(f"No rows found in {related_entity}. Skipping node creation.")
                else:
                    for row in related_rows:
                        rel_id = row[0]
                        node_id = f"{related_entity}_{rel_id}"
                        graph["nodes"].append({"id": node_id, "type": related_entity.lower(), "entity_id": rel_id})
                        print(f"Added node: {node_id}")
            except sqlite3.Error as e:
                print(f"Error querying {related_entity}: {e}")

        edges_added = False
        if transaction_entity:
            for table, details in self.schema.items():
                foreign_keys = details[-1]["foreign_keys"]
                links_to_primary = any(fk["to_table"] == primary_entity for fk in foreign_keys)
                links_to_transaction = any(fk["to_table"] == transaction_entity for fk in foreign_keys)
                if links_to_primary and links_to_transaction and table != primary_entity and table != transaction_entity:
                    print(f"Found linking table: {table}")
                    fk_to_primary = next(fk for fk in foreign_keys if fk["to_table"] == primary_entity)
                    fk_to_transaction = next(fk for fk in foreign_keys if fk["to_table"] == transaction_entity)
                    try:
                        query = f"""
                        SELECT {fk_to_transaction['from']}, {fk_to_primary['from']}
                        FROM {table}
                        WHERE {fk_to_transaction['from']} IS NOT NULL AND {fk_to_primary['from']} IS NOT NULL
                        """
                        self.sql_transformer.cursor.execute(query)
                        edges = self.sql_transformer.cursor.fetchall()
                        if not edges:
                            print(f"No edges found in {table} linking {transaction_entity} to {primary_entity}.")
                        else:
                            for trans_id, entity_id in edges:
                                source_id = f"{transaction_entity}_{trans_id}"
                                target_id = f"{primary_entity}_{entity_id}"
                                if source_id in [node["id"] for node in graph["nodes"]] and target_id in [node["id"] for node in graph["nodes"]]:
                                    graph["edges"].append({"source": source_id, "target": target_id, "type": "link"})
                                    print(f"Added edge: {source_id} -> {target_id}")
                                    edges_added = True
                    except sqlite3.Error as e:
                        print(f"Error creating edges via {table}: {e}")

                if table == transaction_entity:
                    for fk in foreign_keys:
                        if fk["to_table"] == primary_entity:
                            try:
                                # Include edge attributes: total and quantity
                                self.sql_transformer.cursor.execute(
                                    f"SELECT {fk['from']}, {transaction_key}, Total, (SELECT SUM(Quantity) FROM InvoiceLine WHERE InvoiceId = {transaction_key}) AS TotalQuantity "
                                    f"FROM {transaction_entity} WHERE {fk['from']} IS NOT NULL"
                                )
                                edges = self.sql_transformer.cursor.fetchall()
                                if not edges:
                                    print(f"No direct edges found for {fk['from']} -> {primary_entity} in {transaction_entity}.")
                                else:
                                    for entity_id, trans_id, total, quantity in edges:
                                        source_id = f"{transaction_entity}_{trans_id}"
                                        target_id = f"{primary_entity}_{entity_id}"
                                        if source_id in [node["id"] for node in graph["nodes"]] and target_id in [node["id"] for node in graph["nodes"]]:
                                            graph["edges"].append({
                                                "source": source_id,
                                                "target": target_id,
                                                "type": "link",
                                                "total": total if total is not None else 0.0,
                                                "quantity": quantity if quantity is not None else 0.0
                                            })
                                            print(f"Added direct edge: {source_id} -> {target_id} with total {total}, quantity {quantity}")
                                            edges_added = True
                            except sqlite3.Error as e:
                                print(f"Error creating direct edges for {fk['from']} -> {primary_entity}: {e}")

        if related_entity and transaction_entity:
            for table, details in self.schema.items():
                for fk in details[-1]["foreign_keys"]:
                    if fk["to_table"] == related_entity and table == transaction_entity:
                        try:
                            self.sql_transformer.cursor.execute(
                                f"SELECT {fk['from']}, {transaction_key} FROM {transaction_entity} WHERE {fk['from']} IS NOT NULL"
                            )
                            edges = self.sql_transformer.cursor.fetchall()
                            if not edges:
                                print(f"No edges found for {fk['from']} -> {related_entity} in {transaction_entity}.")
                            else:
                                for rel_id, trans_id in edges:
                                    source_id = f"{related_entity}_{rel_id}"
                                    target_id = f"{transaction_entity}_{trans_id}"
                                    if source_id in [node["id"] for node in graph["nodes"]] and target_id in [node["id"] for node in graph["nodes"]]:
                                        graph["edges"].append({"source": source_id, "target": target_id, "type": "link"})
                                        print(f"Added edge: {source_id} -> {target_id}")
                                        edges_added = True
                        except sqlite3.Error as e:
                            print(f"Error creating edges for {fk['from']} -> {related_entity}: {e}")

        if not edges_added:
            print("No edges inferred via foreign keys. Adding fallback edges to table nodes...")
            for node in graph["nodes"]:
                if node["type"] != "table":
                    entity_type = node["type"]
                    table_id = next((table for table in self.schema.keys() if table.lower() == entity_type), None)
                    if table_id and node["id"] != table_id:
                        graph["edges"].append({"source": node["id"], "target": table_id, "type": "belongs_to"})
                        print(f"Added fallback edge: {node.get('id')} -> {table_id}")  # Fixed line

        print(f"Graph built with {len(graph['nodes'])} nodes and {len(graph['edges'])} edges.")
        return graph

    def _prepare_gnn_data(self) -> Data:
        print("Preparing GNN data...")
        nodes = [node["id"] for node in self.graph["nodes"]]
        n = len(nodes)
        edge_index = []
        edge_attr = []
        for edge in self.graph["edges"]:
            src_idx = nodes.index(edge["source"])
            tgt_idx = nodes.index(edge["target"])
            edge_index.append([src_idx, tgt_idx])
            # Use total and quantity as edge attributes
            total = edge.get("total", 0.0)
            quantity = edge.get("quantity", 0.0)
            # Normalize edge attributes
            max_total = max([e.get("total", 0.0) for e in self.graph["edges"]] or [1.0])
            max_quantity = max([e.get("quantity", 0.0) for e in self.graph["edges"]] or [1.0])
            norm_total = total / max_total if max_total > 0 else 0.0
            norm_quantity = quantity / max_quantity if max_quantity > 0 else 0.0
            edge_attr.append([norm_total, norm_quantity])
        edge_index = torch.tensor(edge_index, dtype=torch.long).t()
        edge_attr = torch.tensor(edge_attr, dtype=torch.float) if edge_attr else torch.zeros((edge_index.shape[1], 2), dtype=torch.float)
        features, labels = self._load_enriched_features()
        x = torch.tensor(features, dtype=torch.float) if features is not None else torch.ones((n, 3), dtype=torch.float)
        y = torch.tensor(labels, dtype=torch.float) if labels is not None else None
        return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

    def _load_enriched_features(self) -> tuple[Optional[np.ndarray], Optional[np.ndarray]]:
        print("Loading enriched features...")
        try:
            feature_dim = 3
            features = np.zeros((len(self.graph["nodes"]), feature_dim))
            labels = np.zeros(len(self.graph["nodes"]))

            primary_entity, related_entity, transaction_entity = self._infer_relevant_entities()
            if not primary_entity:
                print("No primary entity identified for feature enrichment.")
                return None, None

            primary_key = next((col["name"] for col in self.schema[primary_entity][:-1] if col["primary_key"]), "id")
            numeric_cols = [col["name"] for col in self.schema[primary_entity][:-1] if col["type"] in ("INTEGER", "REAL", "NUMERIC") and not col["primary_key"]]
            feature_col = numeric_cols[0] if numeric_cols else None

            if transaction_entity and feature_col:
                trans_key = next((col["name"] for col in self.schema[transaction_entity][:-1] if col["primary_key"]), "id")
                for table, details in self.schema.items():
                    foreign_keys = details[-1]["foreign_keys"]
                    links_to_primary = any(fk["to_table"] == primary_entity for fk in foreign_keys)
                    links_to_transaction = any(fk["to_table"] == transaction_entity for fk in foreign_keys)
                    if links_to_primary and links_to_transaction and table != primary_entity and table != transaction_entity:
                        fk_to_primary = next(fk for fk in foreign_keys if fk["to_table"] == primary_entity)
                        fk_to_transaction = next(fk for fk in foreign_keys if fk["to_table"] == transaction_entity)
                        numeric_cols = [col["name"] for col in self.schema[table][:-1] if col["type"] in ("INTEGER", "REAL", "NUMERIC") and col["name"] not in (fk_to_primary["from"], fk_to_transaction["from"])]
                        agg_col = numeric_cols[0] if numeric_cols else None
                        if agg_col:
                            try:
                                query = f"""
                                SELECT {fk_to_primary['from']}, SUM({agg_col}) AS TotalValue
                                FROM {table}
                                GROUP BY {fk_to_primary['from']}
                                """
                                df = pd.read_sql_query(query, self.sql_transformer.conn)
                                for node in self.graph["nodes"]:
                                    if node["type"] == primary_entity.lower():
                                        entity_id = node["entity_id"]
                                        idx = self.node_to_idx[node["id"]]
                                        row = df[df[fk_to_primary["from"]] == entity_id]
                                        if not row.empty:
                                            value = row["TotalValue"].iloc[0] if not pd.isna(row["TotalValue"].iloc[0]) else 0
                                            features[idx] = [value, 0, 0]
                                            labels[idx] = max(0, value)
                            except sqlite3.Error as e:
                                print(f"Error loading features for {primary_entity} via {table}: {e}")

            if related_entity:
                related_key = next((col["name"] for col in self.schema[related_entity][:-1] if col["primary_key"]), "id")
                numeric_cols = [col["name"] for col in self.schema[related_entity][:-1] if col["type"] in ("INTEGER", "REAL", "NUMERIC") and not col["primary_key"]]
                feature_col = numeric_cols[0] if numeric_cols else None
                if feature_col:
                    try:
                        query = f"""
                        SELECT {related_key}, {feature_col}
                        FROM {related_entity}
                        """
                        df = pd.read_sql_query(query, self.sql_transformer.conn)
                        for node in self.graph["nodes"]:
                            if node["type"] == related_entity.lower():
                                entity_id = node["entity_id"]
                                idx = self.node_to_idx[node["id"]]
                                row = df[df[related_key] == entity_id]
                                if not row.empty:
                                    value = row[feature_col].iloc[0] if not pd.isna(row[feature_col].iloc[0]) else 0
                                    features[idx] = [value, 0, 0]
                                    labels[idx] = max(0, value)
                    except sqlite3.Error as e:
                        print(f"Error loading features for {related_entity}: {e}")

            if transaction_entity:
                trans_key = next((col["name"] for col in self.schema[transaction_entity][:-1] if col["primary_key"]), "id")
                numeric_cols = [col["name"] for col in self.schema[transaction_entity][:-1] if col["type"] in ("INTEGER", "REAL", "NUMERIC") and not col["primary_key"]]
                date_cols = [col["name"] for col in self.schema[transaction_entity][:-1] if "date" in col["name"].lower()]
                feature_col = numeric_cols[0] if numeric_cols else None
                date_col = date_cols[0] if date_cols else None
                if feature_col and date_col:
                    try:
                        query = f"""
                        SELECT {trans_key}, {feature_col}, strftime('%Y', {date_col}) AS Year
                        FROM {transaction_entity}
                        """
                        df = pd.read_sql_query(query, self.sql_transformer.conn)
                        for node in self.graph["nodes"]:
                            if node["type"] == transaction_entity.lower():
                                trans_id = node["entity_id"]
                                idx = self.node_to_idx[node["id"]]
                                row = df[df[trans_key] == trans_id]
                                if not row.empty:
                                    value = row[feature_col].iloc[0]
                                    year = float(row["Year"].iloc[0]) if row["Year"].iloc[0] else 0
                                    features[idx] = [value, year, 0]  # Ensure year is populated
                                    labels[idx] = max(0, value)
                    except sqlite3.Error as e:
                        print(f"Error loading features for {transaction_entity}: {e}")

            max_features = np.max(features, axis=0, keepdims=True)
            max_features[max_features == 0] = 1
            features = features / max_features

            max_labels = np.max(labels) if np.max(labels) > 0 else 1
            labels = labels / max_labels

            print("Feature Stats:", np.min(features, axis=0), np.max(features, axis=0))
            print("Label Stats:", np.min(labels), np.max(labels))
            return features, labels
        except Exception as e:
            print(f"Error loading enriched features: {e}")
            return None, None

    def train_gnn(self):
        print("Training GraphSAGE with GAT...")
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = GraphSAGE(input_dim=3, hidden_dim=64, embedding_dim=16, output_dim=1, edge_dim=2, heads=4).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)
        data = self.gnn_data.to(device)

        if data.y is not None:
            labels = data.y.unsqueeze(1)
            for epoch in range(300):
                model.train()
                optimizer.zero_grad()
                predictions, embeddings = model(data)
                loss = F.mse_loss(predictions, labels)
                loss.backward()
                optimizer.step()
                scheduler.step()
                if epoch % 50 == 0:
                    print(f"Epoch {epoch}, Loss: {loss.item()}, LR: {scheduler.get_last_lr()[0]}")
            print(f"Training completed. Final loss: {loss.item()}")
        else:
            print("No labels available; using dummy loss.")
            for epoch in range(300):
                model.train()
                optimizer.zero_grad()
                predictions, embeddings = model(data)
                loss = torch.mean((predictions - data.x) ** 2)
                loss.backward()
                optimizer.step()
                scheduler.step()
            print(f"Training completed. Final loss: {loss.item()}")

        model.eval()
        with torch.no_grad():
            predictions, embeddings = model(data)
            predictions = predictions.cpu().numpy().flatten()
            embeddings = embeddings.cpu().numpy()

        predictions_normalized = np.clip(predictions, 0, None)
        max_pred = np.max(predictions_normalized) if np.max(predictions_normalized) > 0 else 1
        predictions_scaled = (predictions_normalized / max_pred) * 100
        return predictions_scaled, embeddings

# ResultsPresenter Class: Visualizes and presents results
class ResultsPresenter:
    def __init__(self, graph: Dict, predictions: np.ndarray, embeddings: np.ndarray, node_to_idx: Dict, analysis_choice: str, sql_transformer: SQLTransformer):
        self.graph = graph
        self.predictions = predictions
        self.embeddings = embeddings
        self.node_to_idx = node_to_idx
        self.analysis_choice = analysis_choice.lower()
        self.sql_transformer = sql_transformer
        self.schema = sql_transformer.schema

        # Initialize FAISS index for efficient similarity search
        embedding_dim = embeddings.shape[1]
        self.index = faiss.IndexFlatL2(embedding_dim)  # L2 distance for similarity
        self.index.add(embeddings.astype(np.float32))  # Add embeddings to the index
        self.node_ids = [node["id"] for node in self.graph["nodes"]]

    def _infer_relevant_entities(self):
        builder = GNNBuilder(self.sql_transformer, self.analysis_choice)
        return builder._infer_relevant_entities()

    def _force_directed_layout(self, nodes, edges, iterations=50):
        """Compute a force-directed layout for nodes and edges."""
        node_ids = [node["id"] for node in nodes]
        pos = {node_id: np.array([np.random.uniform(-1, 1), np.random.uniform(-1, 1)]) for node_id in node_ids}
        k = 1.0 / np.sqrt(len(nodes))  # Optimal distance between nodes

        for _ in range(iterations):
            # Repulsion between all pairs of nodes
            for i, node1 in enumerate(node_ids):
                for j, node2 in enumerate(node_ids):
                    if i != j:
                        delta = pos[node1] - pos[node2]
                        dist = np.linalg.norm(delta)
                        if dist > 0:
                            repulsion = k * k / dist
                            pos[node1] += (delta / dist) * repulsion
                            pos[node2] -= (delta / dist) * repulsion

            # Attraction along edges
            for edge in edges:
                src = edge["source"]
                tgt = edge["target"]
                if src in pos and tgt in pos:
                    delta = pos[src] - pos[tgt]
                    dist = np.linalg.norm(delta)
                    if dist > 0:
                        attraction = (dist * dist) / k
                        pos[src] -= (delta / dist) * attraction * 0.01
                        pos[tgt] += (delta / dist) * attraction * 0.01

        # Normalize positions
        all_pos = np.array(list(pos.values()))
        if len(all_pos) > 0:
            min_pos = all_pos.min(axis=0)
            max_pos = all_pos.max(axis=0)
            if (max_pos - min_pos).max() > 0:
                all_pos = (all_pos - min_pos) / (max_pos - min_pos) * 10 - 5
        return {node_id: all_pos[i] for i, node_id in enumerate(node_ids)}

    def _circular_layout(self, nodes, clusters):
        """Compute a circular layout, grouping nodes by cluster."""
        node_ids = [node["id"] for node in nodes]
        pos = {}
        n_clusters = len(set(clusters))
        radius = 5
        for cluster in range(n_clusters):
            cluster_nodes = [node_ids[i] for i in range(len(node_ids)) if clusters[i] == cluster]
            for idx, node_id in enumerate(cluster_nodes):
                angle = 2 * np.pi * idx / len(cluster_nodes) + (cluster * 2 * np.pi / n_clusters)
                pos[node_id] = np.array([radius * np.cos(angle), radius * np.sin(angle)])
        return pos

    def present_results(self):
        print("Presenting results...")
        primary_entity, _, _ = self._infer_relevant_entities()
        entity_indices = [i for i, node in enumerate(self.graph["nodes"]) if node["type"] == primary_entity.lower()]
        entity_ids = [self.graph["nodes"][i]["id"] for i in entity_indices]
        entity_names = []

        # Fetch names for primary entities with error handling
        if primary_entity and ("predict " in self.analysis_choice or "analyze " in self.analysis_choice):
            primary_key = next((col["name"] for col in self.schema[primary_entity][:-1] if col["primary_key"]), "id")
            name_col = next((col["name"] for col in self.schema[primary_entity][:-1] if "name" in col["name"].lower()), primary_key)
            try:
                query = f"SELECT {primary_key}, {name_col} FROM {primary_entity}"
                df = pd.read_sql_query(query, self.sql_transformer.conn)
                for node in self.graph["nodes"]:
                    if node["type"] == primary_entity.lower():
                        entity_id = node["entity_id"]
                        row = df[df[primary_key] == entity_id]
                        if not row.empty:
                            node["name"] = row[name_col].iloc[0]
                entity_names = [self.graph["nodes"][i].get("name", self.graph["nodes"][i]["id"]) for i in entity_indices]
            except sqlite3.Error as e:
                print(f"Error fetching names for {primary_entity}: {e}")
                entity_names = [self.graph["nodes"][i]["id"] for i in entity_indices]

            # Top 5 Predictions
            entity_predictions = [
                (entity_ids[i], entity_names[i], self.predictions[entity_indices[i]])
                for i in range(len(entity_indices))
            ]
            entity_predictions.sort(key=lambda x: x[2], reverse=True)
            print(f"\nTop 5 Predicted {primary_entity}s:")
            for entity_id, entity_name, scaled_pred in entity_predictions[:5]:
                print(f"{primary_entity}: {entity_id} ({entity_name}), Scaled Prediction: {scaled_pred:.2f}")

            # Visualization 1: Top Customers and Their Invoices
            print("\nGenerating Visualization 1: Top Customers and Their Invoices...")
            try:
                top_entities = [ep[0] for ep in entity_predictions[:5]]
                subgraph_nodes = []
                subgraph_edges = []
                node_types = {}
                node_predictions = {}
                node_names = {}

                # Add top entities
                for idx, node in enumerate(self.graph["nodes"]):
                    if node["id"] in top_entities:
                        subgraph_nodes.append(node)
                        node_types[node["id"]] = node["type"]
                        node_predictions[node["id"]] = self.predictions[idx]
                        node_names[node["id"]] = node.get("name", node["id"])

                # Add related invoices and tracks
                for edge in self.graph["edges"]:
                    if edge["source"] in top_entities or edge["target"] in top_entities:
                        src_node = next(n for n in self.graph["nodes"] if n["id"] == edge["source"])
                        tgt_node = next(n for n in self.graph["nodes"] if n["id"] == edge["target"])
                        if src_node not in subgraph_nodes:
                            subgraph_nodes.append(src_node)
                            idx = self.node_ids.index(src_node["id"])
                            node_types[src_node["id"]] = src_node["type"]
                            node_predictions[src_node["id"]] = self.predictions[idx]
                            node_names[src_node["id"]] = src_node.get("name", src_node["id"])
                        if tgt_node not in subgraph_nodes:
                            subgraph_nodes.append(tgt_node)
                            idx = self.node_ids.index(tgt_node["id"])
                            node_types[tgt_node["id"]] = tgt_node["type"]
                            node_predictions[tgt_node["id"]] = self.predictions[idx]
                            node_names[tgt_node["id"]] = tgt_node.get("name", tgt_node["id"])
                        subgraph_edges.append(edge)

                # Add tracks from InvoiceLine
                invoice_ids = [node["id"] for node in subgraph_nodes if node["type"] == "invoice"]
                for invoice_id in invoice_ids:
                    invoice_num = int(invoice_id.split("_")[1])
                    try:
                        query = f"""
                        SELECT TrackId FROM InvoiceLine WHERE InvoiceId = {invoice_num}
                        """
                        df = pd.read_sql_query(query, self.sql_transformer.conn)
                        for track_id in df["TrackId"]:
                            track_node_id = f"Track_{track_id}"
                            if track_node_id not in [node["id"] for node in subgraph_nodes]:
                                track_node = {"id": track_node_id, "type": "track", "entity_id": track_id}
                                subgraph_nodes.append(track_node)
                                idx = self.node_ids.index(track_node_id) if track_node_id in self.node_ids else len(self.node_ids)
                                node_types[track_node_id] = "track"
                                node_predictions[track_node_id] = self.predictions[idx] if idx < len(self.predictions) else 0
                                # Fetch track name
                                track_query = f"SELECT Name FROM Track WHERE TrackId = {track_id}"
                                track_df = pd.read_sql_query(track_query, self.sql_transformer.conn)
                                track_name = track_df["Name"].iloc[0] if not track_df.empty else track_node_id
                                node_names[track_node_id] = track_name
                            subgraph_edges.append({"source": invoice_id, "target": track_node_id, "type": "contains"})
                    except sqlite3.Error as e:
                        print(f"Error fetching tracks for invoice {invoice_id}: {e}")

                # Compute layout
                pos = self._force_directed_layout(subgraph_nodes, subgraph_edges)
                edge_x = []
                edge_y = []
                for edge in subgraph_edges:
                    src_pos = pos.get(edge["source"])
                    tgt_pos = pos.get(edge["target"])
                    if src_pos is not None and tgt_pos is not None:
                        edge_x.extend([src_pos[0], tgt_pos[0], None])
                        edge_y.extend([src_pos[1], tgt_pos[1], None])

                node_x = []
                node_y = []
                node_text = []
                node_colors = []
                max_pred = max(node_predictions.values()) if node_predictions else 1
                for node in subgraph_nodes:
                    node_id = node["id"]
                    node_pos = pos.get(node_id)
                    if node_pos is not None:
                        node_x.append(node_pos[0])
                        node_y.append(node_pos[1])
                        pred = node_predictions.get(node_id, 0)
                        scaled_pred = (pred / max_pred) * 100 if max_pred > 0 else 0
                        node_text.append(f"{node_names.get(node_id, node_id)} ({node_types.get(node_id).capitalize()})<br>Prediction: {scaled_pred:.2f}")
                        green_value = int(255 * (pred / max_pred)) if max_pred > 0 else 0
                        node_colors.append(f'rgb(0, {green_value}, 0)')

                fig1 = go.Figure()
                fig1.add_trace(go.Scatter(
                    x=edge_x, y=edge_y,
                    line=dict(width=1, color='#888'),
                    hoverinfo='none',
                    mode='lines',
                    name='Edges'
                ))
                fig1.add_trace(go.Scatter(
                    x=node_x, y=node_y,
                    mode='markers+text',
                    hoverinfo='text',
                    marker=dict(size=15, color=node_colors, line=dict(width=1, color='black')),
                    text=[n.split("<br>")[0] for n in node_text],
                    textposition="top center",
                    hovertext=node_text,
                    name='Nodes'
                ))
                fig1.update_layout(
                    title='Top Customers, Their Invoices, and Purchased Tracks',
                    showlegend=True,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    xaxis=dict(showgrid=False, zeroline=False, title=''),
                    yaxis=dict(showgrid=False, zeroline=False, title=''),
                    width=800,
                    height=600
                )
                fig1.show()
            except Exception as e:
                print(f"Error generating Visualization 1: {e}")

        # Visualization 2: Customer Clusters
        if len(entity_indices) > 1:
            print("\nGenerating Visualization 2: Customer Clusters...")
            try:
                entity_embeddings = self.embeddings[entity_indices]
                kmeans = KMeans(n_clusters=min(3, len(entity_indices)), random_state=42)
                entity_clusters = kmeans.fit_predict(entity_embeddings)

                # Build subgraph for customers
                subgraph_nodes = [self.graph["nodes"][i] for i in entity_indices]
                subgraph_edges = []
                similarities = []
                k = min(4, len(entity_indices))
                distances, indices = self.index.search(entity_embeddings, k)
                for i in range(len(entity_indices)):
                    for j in range(1, k):
                        neighbor_global_idx = indices[i][j]
                        if neighbor_global_idx >= 0 and neighbor_global_idx < len(self.embeddings):
                            neighbor_entity_idx = entity_indices.index(neighbor_global_idx) if neighbor_global_idx in entity_indices else -1
                            if neighbor_entity_idx >= 0 and neighbor_entity_idx != i:
                                distance = distances[i][j]
                                similarity = 1 / (1 + distance) if distance > 0 else 0
                                if similarity > 0.5:
                                    subgraph_edges.append({
                                        "source": entity_ids[i],
                                        "target": entity_ids[neighbor_entity_idx],
                                        "type": "similarity",
                                        "similarity": similarity
                                    })
                                    similarities.append(similarity)

                # Compute layout
                pos = self._circular_layout(subgraph_nodes, entity_clusters)
                edge_x = []
                edge_y = []
                edge_text = []
                for edge in subgraph_edges:
                    src_pos = pos.get(edge["source"])
                    tgt_pos = pos.get(edge["target"])
                    if src_pos is not None and tgt_pos is not None:
                        edge_x.extend([src_pos[0], tgt_pos[0], None])
                        edge_y.extend([src_pos[1], tgt_pos[1], None])
                        edge_text.append(f"Similarity: {edge['similarity']:.2f}")

                node_x = []
                node_y = []
                node_text = []
                node_colors = []
                for idx, node in enumerate(subgraph_nodes):
                    node_id = node["id"]
                    node_pos = pos.get(node_id)
                    if node_pos is not None:
                        node_x.append(node_pos[0])
                        node_y.append(node_pos[1])
                        cluster = entity_clusters[idx]
                        node_text.append(f"{entity_names[idx]} ({node_id})<br>Cluster: {cluster}")
                        node_colors.append(['#FF9999', '#66B2FF', '#99FF99'][cluster % 3])

                fig2 = go.Figure()
                fig2.add_trace(go.Scatter(
                    x=edge_x, y=edge_y,
                    line=dict(width=1, color='#888'),
                    hoverinfo='text',
                    hovertext=edge_text,
                    mode='lines',
                    name='Similarity Edges'
                ))
                fig2.add_trace(go.Scatter(
                    x=node_x, y=node_y,
                    mode='markers+text',
                    hoverinfo='text',
                    marker=dict(size=15, color=node_colors, line=dict(width=1, color='black')),
                    text=[n.split("<br>")[0] for n in node_text],
                    textposition="top center",
                    hovertext=node_text,
                    name='Customers'
                ))
                fig2.update_layout(
                    title='Customer Clusters Based on Purchase Behavior',
                    showlegend=True,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    xaxis=dict(showgrid=False, zeroline=False, title=''),
                    yaxis=dict(showgrid=False, zeroline=False, title=''),
                    width=800,
                    height=600
                )
                fig2.show()
            except Exception as e:
                print(f"Error generating Visualization 2: {e}")

            # Visualization 3: Cross-Selling Opportunities
            print("\nGenerating Visualization 3: Cross-Selling Opportunities...")
            try:
                # Build subgraph for top similar pairs
                top_pairs = sorted(subgraph_edges, key=lambda x: x["similarity"], reverse=True)[:5]
                customer_ids = set()
                for pair in top_pairs:
                    customer_ids.add(pair["source"])
                    customer_ids.add(pair["target"])

                subgraph_nodes = [node for node in self.graph["nodes"] if node["id"] in customer_ids]
                subgraph_edges = [edge for edge in top_pairs]
                track_nodes = set()
                for customer_id in customer_ids:
                    customer_num = int(customer_id.split("_")[1])
                    try:
                        query = f"""
                        SELECT il.TrackId, t.Name
                        FROM Invoice i
                        JOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId
                        JOIN Track t ON il.TrackId = t.TrackId
                        WHERE i.CustomerId = {customer_num}
                        """
                        df = pd.read_sql_query(query, self.sql_transformer.conn)
                        for _, row in df.iterrows():
                            track_id = f"Track_{row['TrackId']}"
                            track_nodes.add((track_id, row["Name"]))
                            subgraph_edges.append({"source": customer_id, "target": track_id, "type": "purchased"})
                    except sqlite3.Error as e:
                        print(f"Error fetching tracks for {customer_id}: {e}")

                for track_id, track_name in track_nodes:
                    subgraph_nodes.append({"id": track_id, "type": "track", "name": track_name})

                # Bipartite layout: customers on left, tracks on right
                pos = {}
                customer_y = np.linspace(-5, 5, len(customer_ids))
                for idx, customer_id in enumerate(customer_ids):
                    pos[customer_id] = np.array([-3, customer_y[idx]])
                track_y = np.linspace(-5, 5, len(track_nodes))
                for idx, (track_id, _) in enumerate(track_nodes):
                    pos[track_id] = np.array([3, track_y[idx]])

                edge_x = []
                edge_y = []
                edge_text = []
                for edge in subgraph_edges:
                    src_pos = pos.get(edge["source"])
                    tgt_pos = pos.get(edge["target"])
                    if src_pos is not None and tgt_pos is not None:
                        edge_x.extend([src_pos[0], tgt_pos[0], None])
                        edge_y.extend([src_pos[1], tgt_pos[1], None])
                        if edge["type"] == "similarity":
                            similarity = edge.get("similarity", 0)
                            edge_text.append(f"Similarity: {similarity:.2f}")
                        else:  # "purchased" type
                            edge_text.append("Purchased")

                node_x = []
                node_y = []
                node_text = []
                node_colors = []
                for node in subgraph_nodes:
                    node_id = node["id"]
                    node_pos = pos.get(node_id)
                    if node_pos is not None:
                        node_x.append(node_pos[0])
                        node_y.append(node_pos[1])
                        node_name = node.get("name", node_id)
                        node_text.append(f"{node_name} ({node['type'].capitalize()})")
                        node_colors.append('#FF9999' if node["type"] == "customer" else '#66B2FF')

                fig3 = go.Figure()
                fig3.add_trace(go.Scatter(
                    x=edge_x, y=edge_y,
                    line=dict(width=1, color='#888'),
                    hoverinfo='text',
                    hovertext=edge_text,
                    mode='lines',
                    name='Edges'
                ))
                fig3.add_trace(go.Scatter(
                    x=node_x, y=node_y,
                    mode='markers+text',
                    hoverinfo='text',
                    marker=dict(size=15, color=node_colors, line=dict(width=1, color='black')),
                    text=[n.split("<br>")[0] for n in node_text],
                    textposition="top center",
                    hovertext=node_text,
                    name='Nodes'
                ))
                fig3.update_layout(
                    title='Cross-Selling Opportunities: Customers and Shared Tracks',
                    showlegend=True,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    xaxis=dict(showgrid=False, zeroline=False, title=''),
                    yaxis=dict(showgrid=False, zeroline=False, title=''),
                    width=800,
                    height=600
                )
                fig3.show()
            except Exception as e:
                print(f"Error generating Visualization 3: {e}")

# Main Workflow
if __name__ == "__main__":
    print("Welcome to the Data Insights Generator!")
    db_path = input("Please specify the path to your SQLite database (e.g., Chinook_Sqlite.sqlite): ").strip()
    if not os.path.exists(db_path):
        print("Database not found! Downloading Chinook database as a default...")
        download_path = os.path.join(os.getcwd(), "Chinook_Sqlite.sqlite")
        os.system(f"wget -O {download_path} https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite")
        if not os.path.exists(download_path):
            raise FileNotFoundError(f"Failed to download Chinook database to {download_path}")
        db_path = download_path
        print(f"Chinook database downloaded to: {db_path}")

    # Step 1: Connect and interrogate data source
    sql_transformer = SQLTransformer(db_path)
    print("\nExtracted Schema:", sql_transformer.schema)

    # Step 2: Use LLM to generate analysis choices
    print("Generating analysis options using LLM...")
    model_name = "gpt2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

    table_names = ", ".join(sql_transformer.schema.keys())
    prompt = (
        f"Based on the following database tables: {table_names}, suggest 4 possible data analysis tasks. "
        "Number them 1 to 4 and describe each task briefly. Focus on sales, customer behavior, or trends. "
        "Example: 1. Predict top-selling products based on sales data. 2. Segment customers by purchase history. "
        "3. Analyze revenue trends over time. 4. Identify cross-selling opportunities between products."
    )
    result = generator(prompt, max_length=200, num_return_sequences=1, temperature=0.7)[0]['generated_text']
    print("\nLLM-generated analysis options:")
    print(result)

    # Parse LLM response to extract options using regex
    analysis_choices = []
    pattern = r"(\d+)\.\s*(.+)"
    matches = re.findall(pattern, result)
    for match in matches[:4]:  # Limit to first 4 matches
        number, description = match
        if int(number) <= 4:
            analysis_choices.append(f"{number}. {description.strip()}")
    if not analysis_choices:
        print("Failed to parse LLM response. Falling back to default choices.")
        analysis_choices = [
            "1. Predict top-selling tracks.",
            "2. Segment customers based on purchase behavior.",
            "3. Identify cross-selling opportunities between tracks.",
            "4. Analyze track sales trends."
        ]
    print("\nAvailable analysis options:")
    for choice in analysis_choices:
        print(choice)

    # Let the user select an analysis choice
    analysis_selection = input("Please select an analysis by number (1-4) or press Enter for default (1): ").strip()
    if not analysis_selection or not analysis_selection.isdigit() or int(analysis_selection) > len(analysis_choices) or int(analysis_selection) < 1:
        analysis_selection = "1"

    selected_analysis = next((choice.split(". ", 1)[1] for choice in analysis_choices if choice.startswith(f"{analysis_selection}.")), analysis_choices[0].split(". ", 1)[1])
    print(f"\nSelected Analysis: {selected_analysis}")

    # Step 3: Build and train the GNN
    gnn_builder = GNNBuilder(sql_transformer, selected_analysis)
    predictions_scaled, embeddings = gnn_builder.train_gnn()

    # Step 4: Present results
    presenter = ResultsPresenter(gnn_builder.graph, predictions_scaled, embeddings, gnn_builder.node_to_idx, selected_analysis, sql_transformer)
    presenter.present_results()

    # Clean up
    sql_transformer.close()

Welcome to the Data Insights Generator!
Please specify the path to your SQLite database (e.g., Chinook_Sqlite.sqlite): Chinook_Sqlite.sqlite
Database not found! Downloading Chinook database as a default...
Chinook database downloaded to: /content/Chinook_Sqlite.sqlite
Connecting to database...
Successfully connected to database at: /content/Chinook_Sqlite.sqlite
Extracting schema...
Found tables: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']

Extracted Schema: {'Album': [{'name': 'AlbumId', 'type': 'INTEGER', 'primary_key': True}, {'name': 'Title', 'type': 'NVARCHAR(160)', 'primary_key': False}, {'name': 'ArtistId', 'type': 'INTEGER', 'primary_key': False}, {'foreign_keys': [{'from': 'ArtistId', 'to_table': 'Artist', 'to_column': 'ArtistId'}]}], 'Artist': [{'name': 'ArtistId', 'type': 'INTEGER', 'primary_key': True}, {'name': 'Name', 'type': 'NVARCHAR(120)', 'primary_key': False}, {'foreign_keys': []}],

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Streaming output truncated to the last 5000 lines.
Added node: InvoiceLine_785
Added node: InvoiceLine_1358
Added node: InvoiceLine_213
Added node: InvoiceLine_1931
Added node: InvoiceLine_786
Added node: InvoiceLine_1359
Added node: InvoiceLine_214
Added node: InvoiceLine_1932
Added node: InvoiceLine_787
Added node: InvoiceLine_1360
Added node: InvoiceLine_215
Added node: InvoiceLine_1933
Added node: InvoiceLine_788
Added node: InvoiceLine_1361
Added node: InvoiceLine_216
Added node: InvoiceLine_1934
Added node: InvoiceLine_789
Added node: InvoiceLine_1362
Added node: InvoiceLine_217
Added node: InvoiceLine_1935
Added node: InvoiceLine_790
Added node: InvoiceLine_1363
Added node: InvoiceLine_218
Added node: InvoiceLine_791
Added node: InvoiceLine_1364
Added node: InvoiceLine_1936
Added node: InvoiceLine_219
Added node: InvoiceLine_1937
Added node: InvoiceLine_1938
Added node: InvoiceLine_1939
Added node: InvoiceLine_792
Added node: InvoiceLine_1940
Added node: InvoiceLine_1941
Added n

<ipython-input-6-4df47569973f>:505: RuntimeWarning: invalid value encountered in multiply
  pos[src] -= (delta / dist) * attraction * 0.01
<ipython-input-6-4df47569973f>:506: RuntimeWarning: invalid value encountered in multiply
  pos[tgt] += (delta / dist) * attraction * 0.01



Generating Visualization 2: Customer Clusters...



Generating Visualization 3: Cross-Selling Opportunities...
